# Dataframe Creation for E11 CH Composition Study

Tamar Ervin
Jan 4, 2024

In [1]:
import glob
import shutil
import pyspedas
from pyspedas import time_string, time_double
from pytplot import tplot, get_data, cdf_to_tplot, store_data
import astrospice
import sunpy 
import sys, os
import datetime
import numpy as np
sys.path.append(os.path.realpath(''))
import pandas as pd
import astropy.units as u
import matplotlib.pyplot as plt

import tools.utilities as utils
import tools.sigma as sigma
import tools.psp_funcs as psp_funcs
import tools.pfss_funcs as pfss_funcs

import sunpy_soar
from sunpy.net import Fido, attrs as a 
import sunpy.coordinates as scoords
from scipy.interpolate import interp1d
from plasmapy.formulary import beta, magnetic_pressure, thermal_pressure, ion_sound_speed
from astropy.constants import k_B
# import heliopy.data.spice as spicedata
# import heliopy.spice as spice
import scipy.constants as con


import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import animation, rc
from IPython.display import HTML
from astropy.coordinates import SkyCoord

import astropy.constants as const
import astropy.units as u
from ntpath import basename
import numpy as np
from urllib.request import urlretrieve
import os 

rc('animation',html='html5')



STORAGE_DIR = '/Volumes/TAMARUSB/pub_data/e11_ch'
RES_DIR = os.path.join(STORAGE_DIR, 'results')
for sc in ['psp','solar orbiter'] : kernels = astrospice.registry.get_kernels(sc,'predict') 



Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

08-Jan-24 10:39:52: /Users/tamarervin/miniconda3/envs/analysis/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),



Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

# PARKER

In [ ]:
# check if folder exists
if not os.path.exists(RES_DIR):
    os.makedirs(RES_DIR)
    print(f"Folder '{RES_DIR}' created.")
else:
    print(f"Folder '{RES_DIR}' already exists.")

In [ ]:
time_range = ['2022-02-23/00:00', '2022-03-01/00:00']
enc = 'E11'
edens = 'ENC11_QTN_Electron_Density.csv'

In [ ]:
# FIELDS
fields_vars = pyspedas.psp.fields(trange=time_range, time_clip=True, datatype='mag_RTN_4_Sa_per_Cyc')

# RFS
rfs_vars = pyspedas.psp.fields(trange=time_range, time_clip=True, level='l3', datatype='rfs_lfr')

# SPAN-Ion - Proton Moments
pvars = pyspedas.psp.spi(trange=time_range, datatype='sf00_l3_mom', 
                            level='l3', time_clip=True)

# SPAN-Ion - Alpha Particle Moments
avars = pyspedas.psp.spi(trange=time_range, datatype='sf0a_l3_mom', 
                            level='l3', time_clip=True)

# SPAN-Electron - Electron Moments
spe_vars = pyspedas.psp.spe(trange=time_range, level='l2', time_clip=True)

In [ ]:
# MOVE DATA TO USB
shutil.move('psp_data', STORAGE_DIR)

### FIELDS

In [ ]:
files = glob.glob(os.path.join(os.path.join(STORAGE_DIR, 'psp_data', 'fields/l2/mag_rtn_4_per_cycle/2022'), "*"), recursive=True)
vars = cdf_to_tplot(files)
dt = get_data('psp_fld_l2_mag_RTN_4_Sa_per_Cyc')
date_obj = [datetime.datetime.strptime(time_string(d), '%Y-%m-%d %H:%M:%S.%f') for d in dt.times]

rd = {'Time': date_obj, 'Br': dt.y[:, 0], 'Bt': dt.y[:, 1], 'Bn': dt.y[:, 2]}
fields = pd.DataFrame(data=rd)
fields.to_csv(os.path.join(RES_DIR, 'fields.csv'))

In [ ]:
tplot(['psp_fld_l2_mag_RTN_4_Sa_per_Cyc'])

### PROTONS

In [ ]:
files = glob.glob(os.path.join(os.path.join(STORAGE_DIR, 'psp_data', 'sweap/spi/l3/spi_sf00_l3_mom/2022'), "*"), recursive=True)

# files = glob.glob(os.path.join(path[0], '*'))

vars = cdf_to_tplot(files)
dt = get_data('VEL_RTN_SUN')
dt2 = get_data('DENS')
dt3 = get_data('TEMP')
date_obj = [datetime.datetime.strptime(time_string(d), '%Y-%m-%d %H:%M:%S.%f') for d in dt.times]

rd = {'Time': date_obj, 'vr': np.abs(dt.y[:, 0]), 'vt': dt.y[:, 1], 'vn': dt.y[:, 2], 'Np': dt2.y, 'Tp': dt3.y}
protons = pd.DataFrame(data=rd)

### ADD ANGLE
vx, vy, vz = [get_data('VEL_SC').y[:, i] for i in np.arange(0, 3)]
mx, my, mz = [get_data('MAGF_INST').y[:, i] for i in np.arange(0, 3)]
vdotb = vx*mx + vy*my + vz*mz
v = np.sqrt(vx**2 + vy**2 + vz**2)
b = np.sqrt(mx**2 + my**2 + mz**2)
angle_vb = np.arccos(vdotb/(v*b))

protons['angle_vb'] = angle_vb

protons.to_csv(os.path.join(RES_DIR, 'protons.csv'))

In [ ]:
tplot(['VEL_INST',
 'VEL_SC',
 'VEL_RTN_SUN',
 'SC_VEL_RTN_SUN'])
vars


In [ ]:
tplot(['DENS', 'VEL_RTN_SUN', 'TEMP', 'EFLUX_VS_THETA', 'EFLUX_VS_PHI'])

### ALPHAS

In [ ]:
files = glob.glob(os.path.join(os.path.join(STORAGE_DIR, 'psp_data', 'sweap/spi/l3/spi_sf0a_l3_mom/2022'), "*"), recursive=True)
# files = glob.glob(os.path.join(path[0], '*'))

vars = cdf_to_tplot(files)
dt = get_data('VEL_RTN_SUN')
dt2 = get_data('DENS')
dt3 = get_data('TEMP')
date_obj = [datetime.datetime.strptime(time_string(d), '%Y-%m-%d %H:%M:%S.%f') for d in dt.times]

rd = {'Time': date_obj, 'vra': dt.y[:, 0], 'vta': dt.y[:, 1], 'vna': dt.y[:, 2], 'Na': dt2.y, 'Ta': dt3.y}
alphas = pd.DataFrame(data=rd)
alphas.to_csv(os.path.join(RES_DIR, 'alphas.csv'))

In [ ]:
tplot(['DENS', 'VEL_RTN_SUN', 'TEMP', 'EFLUX_VS_THETA', 'EFLUX_VS_PHI'])

### ELECTRONS

In [ ]:
dt = pd.read_csv('/Users/tamarervin/publications/ENC11_SPAN-E_Core_Fits.csv')
date_obj = [datetime.datetime.strptime(d, '%Y-%m-%d/%H:%M:%S') for d in dt.Time]

rd = {'Time': date_obj, 'Ne': dt.Ne, 'Te': dt['T']}
electrons = pd.DataFrame(data=rd, index=None)
# dfe = dfe.set_index(dfe.Time)
electrons.to_csv(os.path.join(RES_DIR, 'electrons.csv'))

### COMBINE DATASETS

In [8]:
protons = pd.read_csv(os.path.join(RES_DIR, 'protons.csv'), index_col=None)
electrons = pd.read_csv(os.path.join(RES_DIR, 'electrons.csv'), index_col=None)
alphas = pd.read_csv(os.path.join(RES_DIR, 'alphas.csv'), index_col=None)
fields = pd.read_csv(os.path.join(RES_DIR, 'fields.csv'), index_col=None)
protons, electrons, alphas, fields = [dd.drop(['Unnamed: 0'], axis=1) for dd in [protons, electrons, alphas, fields]]
protons['Time'] = [datetime.datetime.strptime(d, '%Y-%m-%d %H:%M:%S.%f') for d in protons.Time]
electrons['Time'] = [datetime.datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in electrons.Time]
alphas['Time'] = [datetime.datetime.strptime(d, '%Y-%m-%d %H:%M:%S.%f') for d in alphas.Time]
fields['Time'] = [datetime.datetime.strptime(d, '%Y-%m-%d %H:%M:%S.%f') for d in fields.Time]


KeyError: "['Unnamed: 0'] not found in axis"

In [9]:
ff = pd.merge_asof(protons, alphas, on='Time', direction='backward')
ff = pd.merge_asof(ff, electrons, on='Time', direction='backward')
ff = pd.merge_asof(ff, fields, on='Time', direction='backward')
ff = ff.set_index('Time')
ff = ff[np.logical_and(ff.Tp > 0, ff.Ta > 0)].copy()
ff

,vr,vt,vn,Np,Tp,angle_vb,vra,vta,vna,Na,Ta,Ne,Te,Br,Bt,Bn
Time,,,,,,,,,,,,,,,,
2022-02-23 00:00:03.926892,370.97455,-68.608260,12.858900,145.62561,86.790770,1.884407,465.81497,-129.249470,-36.253708,2.197340,964.12860,185.252502,33.880196,-95.51137,13.189177,7.895578
2022-02-23 00:00:07.422132,382.27350,-74.660080,14.060577,161.48608,89.291084,1.826346,462.15216,-120.491400,3.749751,2.321620,498.29004,185.252502,33.880196,-96.65181,11.872073,-3.645160
2022-02-23 00:00:10.917373,364.42395,-74.207940,19.968895,177.64952,89.371605,1.865545,403.23470,-56.667350,-62.046680,1.500966,607.38560,185.252502,33.880196,-93.71363,-6.216879,-5.682634
2022-02-23 00:00:14.412674,372.37628,-75.410360,28.382940,169.79265,79.205550,1.941305,530.37024,-74.644510,-9.971888,2.041062,706.98627,185.252502,33.880196,-95.36170,2.155807,7.981667
2022-02-23 00:00:17.907915,374.23737,-78.762375,24.308477,158.62505,88.632700,1.909643,506.34766,-103.938210,-22.601944,2.155727,671.91187,185.252502,33.880196,-94.05744,-2.235075,3.259651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-28 23:59:43.194114,228.82224,-8.253631,-10.827599,276.13450,18.814621,1.398025,273.78880,-24.720728,-10.724208,9.546747,119.24262,227.877926,27.971711,84.80552,-8.726909,15.746349
2022-02-28 23:59:46.689415,229.35385,-9.343748,-10.732706,279.30658,18.966917,1.340095,267.28770,-34.630830,-1.861496,9.224733,129.30023,227.877926,27.971711,85.19100,-2.531784,12.523057
2022-02-28 23:59:50.184655,228.71349,-10.778193,-9.042365,282.03693,18.999338,1.329281,260.90866,-23.886229,-5.436355,7.178066,128.28258,227.877926,27.971711,85.53899,-3.666060,11.854454


In [10]:
# Calculate time differences between consecutive datetime values
from datetime import timedelta
datetime_list = ff.index.to_list()
time_diff_list = [datetime_list[i + 1] - datetime_list[i] for i in range(len(datetime_list) - 1)]

# Calculate the average time difference
average_time_difference = sum(time_diff_list, timedelta()) / len(time_diff_list)
print(average_time_difference)

# Specify the total time period (e.g., 10 minutes)
total_time_period = timedelta(minutes=20)

# Calculate the number of points
number_of_points = total_time_period / average_time_difference

print(f"Number of Points in {total_time_period}: {int(number_of_points)}")


0 days 00:00:02.555789047
Number of Points in 0:20:00: 469


In [11]:
### ----- CALCULATIONS ----- ###
### Vap
ff['B'] = np.sqrt(ff.Br**2 + ff.Bt**2 + ff.Bn**2)
cost = np.abs(ff.Br/ff.B)
ff['vap'] = (ff.vra - ff.vr)/cost

### Ahe
ff['Ahe'] = ff.Na/ff.Np

### TUBULENCE
ff['use_dens'] = ff.Np
ff['sigmac'],ff['sigmar'], ff['vA'], ff['Zp'],ff['Zm'], ff['deltav'], ff['deltab'] = sigma.calc_sigma(ff, num=469)
ff = ff.drop(['use_dens'], axis=1)
ff['diff'] = np.abs(ff.vap)/ff.vA

### MACH NUMBER
ff['MA'] = ff.vr / ff.vA
sound_speed = ion_sound_speed(
    T_e=np.array(ff.Te)*u.eV/k_B,
    T_i=np.array(ff.Tp)*u.eV/k_B,
    # n_e=np.array(ff.Ne)/(u.cm**3),
    # k=k_2,
    ion='p',
    gamma_e=1,
    gamma_i=3,
).to(u.km/u.s).value
ff['MS'] = ff.vr / sound_speed
ff['MMS'] = ff.vr / np.sqrt(sound_speed**2 + ff.vA**2)

### BETA
ff['beta'] = beta(np.array(ff.Tp)*u.eV, np.array(ff.Np)/(u.cm*u.cm*u.cm), np.array(ff.B)*u.nT).value
ff['betae'] = beta(np.array(ff.Te)*u.eV, np.array(ff.Ne)/(u.cm*u.cm*u.cm), np.array(ff.B)*u.nT).value

# ### MAGNETIC PRESSURE
ff['Pm'] = magnetic_pressure(np.array(ff.B)*u.nT).value

### PROTON PRESSURE
ff['Pp'] = thermal_pressure(np.array(ff.Tp)*u.eV, np.array(ff.Np)/u.cm**3).value

### ALPHA PRESSURE
ff['Pa'] = thermal_pressure(np.array(ff.Ta)*u.eV, np.array(ff.Na)/u.cm**3).value

### ELECTRON PRESSURE
ff['Pe'] = thermal_pressure(np.array(ff.Te)*u.eV, np.array(ff.Ne)/u.cm**3).value

### PARKER DF
parker = ff.copy()
parker

,vr,vt,vn,Np,Tp,angle_vb,vra,vta,vna,Na,...,diff,MA,MS,MMS,beta,betae,Pm,Pp,Pa,Pe
Time,,,,,,,,,,,,,,,,,,,,,
2022-02-23 00:00:03.926892,370.97455,-68.608260,12.858900,145.62561,86.790770,1.884407,465.81497,-129.249470,-36.253708,2.197340,...,0.547483,2.114307,2.209672,1.527642,0.543808,0.270050,3.723715e-09,2.024984e-09,3.394241e-10,1.005589e-09
2022-02-23 00:00:07.422132,382.27350,-74.660080,14.060577,161.48608,89.291084,1.826346,462.15216,-120.491400,3.749751,2.321620,...,0.479846,2.277661,2.248494,1.600138,0.611451,0.266151,3.778261e-09,2.310221e-09,1.853462e-10,1.005589e-09
2022-02-23 00:00:10.917373,364.42395,-74.207940,19.968895,177.64952,89.371605,1.865545,403.23470,-56.667350,-62.046680,1.500966,...,0.252200,2.358590,2.142647,1.585941,0.722128,0.285470,3.522571e-09,2.543747e-09,1.460649e-10,1.005589e-09
2022-02-23 00:00:14.412674,372.37628,-75.410360,28.382940,169.79265,79.205550,1.941305,530.37024,-74.644510,-9.971888,2.041062,...,0.986368,2.316086,2.309103,1.635246,0.591051,0.275842,3.645527e-09,2.154690e-09,2.311945e-10,1.005589e-09
2022-02-23 00:00:17.907915,374.23737,-78.762375,24.308477,158.62505,88.632700,1.909643,506.34766,-103.938210,-22.601944,2.155727,...,0.808244,2.287547,2.208466,1.588843,0.638798,0.285173,3.526246e-09,2.252559e-09,2.320686e-10,1.005589e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-28 23:59:43.194114,228.82224,-8.253631,-10.827599,276.13450,18.814621,1.398025,273.78880,-24.720728,-10.724208,9.546747,...,0.402568,2.003900,2.544665,1.574344,0.278340,0.341492,2.990554e-09,8.323894e-10,1.823884e-10,1.021249e-09
2022-02-28 23:59:46.689415,229.35385,-9.343748,-10.732706,279.30658,18.966917,1.340095,267.28770,-34.630830,-1.861496,9.224733,...,0.340006,2.032994,2.543703,1.588101,0.287462,0.345879,2.952620e-09,8.487666e-10,1.911012e-10,1.021249e-09
2022-02-28 23:59:50.184655,228.71349,-10.778193,-9.042365,282.03693,18.999338,1.329281,260.90866,-23.886229,-5.436355,7.178066,...,0.288797,2.030355,2.535148,1.584759,0.288817,0.343557,2.972571e-09,8.585287e-10,1.475318e-10,1.021249e-09


In [12]:
### FLAG DATA

# check if there is any alpha data
all_nan = parker['Ahe'].isna().all()
parker['flag'] = np.ones(len(parker.Ahe))
if all_nan:
    parker['flag'] = np.zeros(len(parker.Ahe))
else:
    # Flag instances as '0' if all three conditions apply
    flag_condition_1 = (np.abs(parker.Np - parker.Ne) / parker.Ne) <= 0.5
    flag_condition_2 = (parker.Ahe) <= 0.2
    flag_condition_3 = (np.abs(parker.vap) / parker.vA) <= 2
    flag_condition_4 = parker.Te < 200 

    flag_0 = np.logical_and.reduce([flag_condition_1, flag_condition_2, flag_condition_3, flag_condition_4])

parker['flag'][flag_0] = 0


In [13]:
### Create SkyCoord for PSP in the inertial (J2000) frame
tt = pd.to_datetime(ff.index.to_list())

psp_inertial = astrospice.generate_coords(
    'SOLAR PROBE PLUS', tt
)

### Transform to solar co-rotating frame 
psp_carrington = psp_inertial.transform_to(
    scoords.HeliographicCarrington(observer="self")
)

# projection
ts_common = np.array([dt.timestamp() for dt in tt])
psp_vr_ts = [int(dt.timestamp()) for dt in tt]
psp_vr_common = interp1d(psp_vr_ts,ff.vr,bounds_error=False)(ts_common)*u.km/u.s
psp_at_source_surface = psp_funcs.ballistically_project(psp_carrington,vr_arr=psp_vr_common, r_inner=2.0*u.R_sun)


In [14]:
### ADD POSITION INFORMAITON AND SAVE
parker['lon'] = psp_carrington.lon.value
parker['lat'] = psp_carrington.lat.value
parker['rAU'] = psp_carrington.radius.to(u.AU).value
parker['sslon'] = psp_at_source_surface.lon.value
parker['sslat'] = psp_at_source_surface.lat.value
parker['ssrAU'] = psp_at_source_surface.radius.to(u.AU).value
parker['NpR2'] = parker.Np * (parker.rAU ** 2)
parker['NeR2'] = parker.Ne * (parker.rAU ** 2)
parker['BrR2'] = parker.Br * (parker.rAU ** 2)
parker.to_csv(os.path.join(RES_DIR, 'parker.csv'))

In [18]:
parker

,vr,vt,vn,Np,Tp,angle_vb,vra,vta,vna,Na,...,flag,lon,lat,rAU,sslon,sslat,ssrAU,NpR2,NeR2,BrR2
Time,,,,,,,,,,,,,,,,,,,,,
2022-02-23 00:00:03.926892,370.97455,-68.608260,12.858900,145.62561,86.790770,1.884407,465.81497,-129.249470,-36.253708,2.197340,...,0.0,47.708430,2.388760,0.156872,57.771146,2.388760,0.009301,3.583668,4.558838,-2.350418
2022-02-23 00:00:07.422132,382.27350,-74.660080,14.060577,161.48608,89.291084,1.826346,462.15216,-120.491400,3.749751,2.321620,...,0.0,47.708403,2.388731,0.156870,57.607543,2.388731,0.009301,3.973891,4.558742,-2.378433
2022-02-23 00:00:10.917373,364.42395,-74.207940,19.968895,177.64952,89.371605,1.865545,403.23470,-56.667350,-62.046680,1.500966,...,0.0,47.708377,2.388702,0.156869,57.972668,2.388702,0.009301,4.371553,4.558645,-2.306081
2022-02-23 00:00:14.412674,372.37628,-75.410360,28.382940,169.79265,79.205550,1.941305,530.37024,-74.644510,-9.971888,2.041062,...,0.0,47.708350,2.388673,0.156867,57.796664,2.388673,0.009301,4.178125,4.558549,-2.346586
2022-02-23 00:00:17.907915,374.23737,-78.762375,24.308477,158.62505,88.632700,1.909643,506.34766,-103.938210,-22.601944,2.155727,...,0.0,47.708324,2.388644,0.156865,57.792816,2.388644,0.009301,3.903239,4.558452,-2.314443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-28 23:59:43.194114,228.82224,-8.253631,-10.827599,276.13450,18.814621,1.398025,273.78880,-24.720728,-10.724208,9.546747,...,0.0,187.868751,0.477085,0.184722,207.395602,0.477085,0.009301,9.422366,7.775737,2.893766
2022-02-28 23:59:46.689415,229.35385,-9.343748,-10.732706,279.30658,18.966917,1.340095,267.28770,-34.630830,-1.861496,9.224733,...,0.0,187.868571,0.477111,0.184724,207.362644,0.477111,0.009301,9.530766,7.775868,2.906969
2022-02-28 23:59:50.184655,228.71349,-10.778193,-9.042365,282.03693,18.999338,1.329281,260.90866,-23.886229,-5.436355,7.178066,...,0.0,187.868391,0.477137,0.184726,207.404395,0.477137,0.009301,9.624097,7.776000,2.918893


# ORBITER

## Data Download

In [ ]:
### TIME RANGE OF INTEREST
timerange = a.Time("2022-02-18", "2022-03-05")

In [ ]:
### SEARCH FOR MAG RTN DATA - L2 MAG
mag_files = Fido.search(timerange, 
                      a.Instrument("MAG"), 
                      a.Level(2), a.Provider.soar, a.soar.Product('mag-rtn-normal-1-minute'))
### SEARCH FOR PAS PARTICLE DATA - L2 SWA
pas_files = Fido.search(timerange, 
                      a.Instrument("SWA"), 
                      a.Level(2), a.Provider.soar, a.soar.Product('swa-pas-grnd-mom'))
### SEARCH FOR HIS DATA - L3 SWA 
his_files = Fido.search(timerange, 
                      a.Instrument("SWA"), 
                      a.Level(3), a.Provider.soar)

In [ ]:
### DOWNLOAD MAG DATA
mag = Fido.fetch(mag_files, path=os.path.join(RES_DIR, "solar_orbiter_data/mag/L2/2022"))

### DOWNLOAD PAS DATA
pas = Fido.fetch(pas_files, path=os.path.join(RES_DIR, "solar_orbiter_data/swa/L2/2022"))

### DOWNLOAD HIS DATA
his = Fido.fetch(his_files, path=os.path.join(RES_DIR, "solar_orbiter_data/swa/L3/2022"))

In [ ]:
### CREATE MAG TIMESERIES
mag_data = sunpy.timeseries.TimeSeries(mag, concatenate=True)

### CREATE MAG TIMESERIES
pas_data = sunpy.timeseries.TimeSeries(pas, concatenate=True)

### CREATE MAG TIMESERIES
his_data = sunpy.timeseries.TimeSeries(his, concatenate=True)

## Create Dataframes

In [ ]:
files = glob.glob(os.path.join(os.path.join(STORAGE_DIR, 'solar_orbiter_data', 'mag/L2/2022'), "*"), recursive=True)
vars = cdf_to_tplot(files)
dt = get_data('B_RTN')
date_obj = [datetime.datetime.strptime(time_string(d), '%Y-%m-%d %H:%M:%S.%f') for d in dt.times]

rd = {'Time': date_obj, 'Br': dt.y[:, 0], 'Bt': dt.y[:, 1], 'Bn': dt.y[:, 2]}
mag = pd.DataFrame(data=rd)
mag.to_csv(os.path.join(RES_DIR, 'mag.csv'))
mag

In [ ]:
files = glob.glob(os.path.join(os.path.join(STORAGE_DIR, 'solar_orbiter_data', 'swa/L2/2022'), "*"), recursive=True)
vars = cdf_to_tplot(files)
dt = get_data('V_RTN')
dt2 = get_data('N')
dt3 = get_data('T')
date_obj = [datetime.datetime.strptime(time_string(d), '%Y-%m-%d %H:%M:%S.%f') for d in dt.times]

rd = {'Time': date_obj, 'vr': dt.y[:, 0], 'vt': dt.y[:, 1], 'vn': dt.y[:, 2], 'Np': dt2.y, 'Tp': dt3.y}
swa = pd.DataFrame(data=rd)
# swa = df.set_index(df.Time)
swa.to_csv(os.path.join(RES_DIR, 'swa.csv'))
swa

In [ ]:
files = glob.glob(os.path.join(os.path.join(STORAGE_DIR, 'solar_orbiter_data', 'swa/L3/2022'), "*"), recursive=True)

vars = cdf_to_tplot(files)

dt = get_data('FE_O_ABUN')
dt2 = get_data('C6_C4_RATIO')
dt3 = get_data('C6_C5_RATIO')
dt4 = get_data('O7_O6_RATIO')
date_obj = [datetime.datetime.strptime(time_string(d), '%Y-%m-%d %H:%M:%S.%f') for d in dt.times]

rd = {'Time': date_obj, 'FeO': dt.y, 'C6C4': dt2.y, 'C6C5': dt3.y, 'O7O6': dt4.y}
his = pd.DataFrame(data=rd)
his.to_csv(os.path.join(RES_DIR, 'his.csv'))

In [ ]:
tplot(['N', 'V_RTN', 'B_RTN', 'T'])


In [ ]:
ff = pd.merge_asof(swa, mag, on='Time', direction='backward')
ff = ff.set_index('Time')
ff

In [ ]:
### Create SkyCoord for PSP in the inertial (J2000) frame
for sc in ['psp','solar orbiter'] : kernels = astrospice.registry.get_kernels(sc,'predict') 

tt = pd.to_datetime(ff.index.to_list())
kernels =  astrospice.registry.get_kernels('solar orbiter','predict') 
solo_inertial = astrospice.generate_coords(
    'SOLAR ORBITER', tt

)

### Transform to solar co-rotating frame 
solo_carrington = solo_inertial.transform_to(
    scoords.HeliographicCarrington(observer="self")
)

# projection
ts_common = np.array([dt.timestamp() for dt in tt])
solo_vr_ts = [int(dt.timestamp()) for dt in tt]
solo_vr_common = interp1d(solo_vr_ts,ff.vr,bounds_error=False)(ts_common)*u.km/u.s
solo_at_source_surface = psp_funcs.ballistically_project(solo_carrington,vr_arr=solo_vr_common, r_inner=2.0*u.R_sun)


In [ ]:
# Calculate time differences between consecutive datetime values
from datetime import timedelta
datetime_list = ff.index.to_list()
time_diff_list = [datetime_list[i + 1] - datetime_list[i] for i in range(len(datetime_list) - 1)]

# Calculate the average time difference
average_time_difference = sum(time_diff_list, timedelta()) / len(time_diff_list)
print(average_time_difference)

# Specify the total time period (e.g., 10 minutes)
total_time_period = timedelta(minutes=20)

# Calculate the number of points
number_of_points = total_time_period / average_time_difference

print(f"Number of Points in {total_time_period}: {int(number_of_points)}")


In [ ]:
### ----- CALCULATIONS ----- ###
### Vap
ff['B'] = np.sqrt(ff.Br**2 + ff.Bt**2 + ff.Bn**2)

### TUBULENCE
ff['use_dens'] = ff.Np
ff['sigmac'],ff['sigmar'], ff['vA'], ff['Zp'],ff['Zm'], ff['deltav'], ff['deltab'] = sigma.calc_sigma(ff, num=283)
ff = ff.drop(['use_dens'], axis=1)

### MACH NUMBER
ff['MA'] = ff.vr / ff.vA
ff['MS'] = np.sqrt((ff.vr)**2 + (ff.vt)**2 + (ff.vn)**2)/con.speed_of_sound

### BETA
ff['beta'] = beta(np.array(ff.Tp)*u.eV, np.array(ff.Np)/(u.cm*u.cm*u.cm), np.array(ff.B)*u.nT).value

# ### MAGNETIC PRESSURE
ff['Pm'] = magnetic_pressure(np.array(ff.B)*u.nT).value

### PROTON PRESSURE
ff['Pp'] = thermal_pressure(np.array(ff.Tp)*u.eV, np.array(ff.Np)/u.cm**3).value

### ORBITER DF
orbiter = ff.copy()

In [ ]:
### ADD POSITION INFORMATION AND SAVE
orbiter['lon'] = solo_carrington.lon.value
orbiter['lat'] = solo_carrington.lat.value
orbiter['rAU'] = solo_carrington.radius.to(u.AU).value
orbiter['sslon'] = solo_at_source_surface.lon.value
orbiter['sslat'] = solo_at_source_surface.lat.value
orbiter['ssrAU'] = solo_at_source_surface.radius.to(u.AU).value
orbiter['NpR2'] = orbiter.Np * (orbiter.rAU ** 2)
orbiter['BrR2'] = orbiter.Br * (orbiter.rAU ** 2)
orbiter.to_csv(os.path.join(RES_DIR, 'orbiter.csv'))
orbiter

### SAME FOR HIS OBS

In [ ]:
ff = pd.merge_asof(mag, swa, on='Time', direction='backward')
ff = pd.merge_asof(his, ff, on='Time', direction='backward')
ff = ff.set_index('Time')
ff

In [ ]:
### Create SkyCoord for PSP in the inertial (J2000) frame
for sc in ['psp','solar orbiter'] : kernels = astrospice.registry.get_kernels(sc,'predict') 

tt = pd.to_datetime(ff.index.to_list())
kernels =  astrospice.registry.get_kernels('solar orbiter','predict') 
solo_inertial = astrospice.generate_coords(
    'SOLAR ORBITER', tt

)

### Transform to solar co-rotating frame 
solo_carrington = solo_inertial.transform_to(
    scoords.HeliographicCarrington(observer="self")
)

# projection
ts_common = np.array([dt.timestamp() for dt in tt])
solo_vr_ts = [int(dt.timestamp()) for dt in tt]
solo_vr_common = interp1d(solo_vr_ts,ff.vr,bounds_error=False)(ts_common)*u.km/u.s
solo_at_source_surface = psp_funcs.ballistically_project(solo_carrington,vr_arr=solo_vr_common, r_inner=2.0*u.R_sun)


In [ ]:
# Calculate time differences between consecutive datetime values
from datetime import timedelta
datetime_list = ff.index.to_list()
time_diff_list = [datetime_list[i + 1] - datetime_list[i] for i in range(len(datetime_list) - 1)]

# Calculate the average time difference
average_time_difference = sum(time_diff_list, timedelta()) / len(time_diff_list)
print(average_time_difference)

# Specify the total time period (e.g., 10 minutes)
total_time_period = timedelta(minutes=20)

# Calculate the number of points
number_of_points = total_time_period / average_time_difference

print(f"Number of Points in {total_time_period}: {int(number_of_points)}")


In [ ]:
### ----- CALCULATIONS ----- ###
### Vap
ff['B'] = np.sqrt(ff.Br**2 + ff.Bt**2 + ff.Bn**2)

### TUBULENCE
ff['use_dens'] = ff.Np
ff['sigmac'],ff['sigmar'], ff['vA'], ff['Zp'],ff['Zm'], ff['deltav'], ff['deltab']  = sigma.calc_sigma(ff, num=8)
ff = ff.drop(['use_dens'], axis=1)

### MACH NUMBER
ff['MA'] = ff.vr / ff.vA
ff['MS'] = np.sqrt((ff.vr)**2 + (ff.vt)**2 + (ff.vn)**2)/con.speed_of_sound

### BETA
ff['beta'] = beta(np.array(ff.Tp)*u.eV, np.array(ff.Np)/(u.cm*u.cm*u.cm), np.array(ff.B)*u.nT).value

# ### MAGNETIC PRESSURE
ff['Pm'] = magnetic_pressure(np.array(ff.B)*u.nT).value

### PROTON PRESSURE
ff['Pp'] = thermal_pressure(np.array(ff.Tp)*u.eV, np.array(ff.Np)/u.cm**3).value

### ORBITER DF
orbiter_his = ff.copy()

In [ ]:
### ADD POSITION INFORMATION AND SAVE
orbiter_his['lon'] = solo_carrington.lon.value
orbiter_his['lat'] = solo_carrington.lat.value
orbiter_his['rAU'] = solo_carrington.radius.to(u.AU).value
orbiter_his['sslon'] = solo_at_source_surface.lon.value
orbiter_his['sslat'] = solo_at_source_surface.lat.value
orbiter_his['ssrAU'] = solo_at_source_surface.radius.to(u.AU).value
orbiter_his['NpR2'] = orbiter.Np * (orbiter.rAU ** 2)
orbiter_his['BrR2'] = orbiter.Br * (orbiter.rAU ** 2)
orbiter_his.to_csv(os.path.join(RES_DIR, 'orbiter_his.csv'))
orbiter_his